# Router Query Engine

여러 후보 쿼리 엔진 중 하나를 선택하여 쿼리를 실행

쿼리를 실행하기 위해 여러 후보 쿼리 엔진 중 하나를 선택하는 사용자 지정 라우터 쿼리 엔진을 정의합니다.

In [1]:
# !export TOKENIZERS_PARALLELISM=false

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import logging
import sys

# enable INFO level logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().handlers = []
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
import nest_asyncio
nest_asyncio.apply()

#### Set Langfuse

In [4]:
from llama_index.core import Settings
from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler

langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key="pk-lf-29339a8f-8a05-4d10-9a0a-a4718f79a53d",
    secret_key="sk-lf-d1f94511-893b-456e-973e-f57b44d50a30",
    host="https://cloud.langfuse.com"
)
Settings.callback_manager = CallbackManager([langfuse_callback_handler])

#### Global Models

In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# embed_model = HuggingFaceEmbedding(
#     model_name="jhgan/ko-sbert-nli",
#     normalize=True,
# )

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

# Settings.llm = OpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)
Settings.llm = OpenAI(model="gpt-4-0613", temperature=0)
# Settings.embed_model = embed_model

#### Load Data
We first show how to convert a Document into a set of Nodes, and insert into a DocumentStore.

In [6]:
"""Load from pdf file"""

from llama_index.readers.file import PyMuPDFReader

root_path = "/Users/heewungsong/Experiment/Visa_Rag"

visa_pdf_file_path = f"{root_path}/Documents/E-9 Visa Guide_한국어.pdf"
visa_docs = PyMuPDFReader().load_data(file_path=visa_pdf_file_path, metadata=True)

faq_pdf_file_path = f"{root_path}/Documents/E-9 FAQ_한국어.pdf"
faq_docs = PyMuPDFReader().load_data(file_path=faq_pdf_file_path, metadata=True)

In [7]:
print(f"Count of faq documents: {len(faq_docs)}")
print(f"Faq metadata: {faq_docs[0].metadata}")
print(f"{faq_docs[0]}\n\n")


print(f"Count of visa documents: {len(visa_docs)}")
print(f"Visa metadata: {visa_docs[0].metadata}")
print(visa_docs[0])

Count of faq documents: 44
Faq metadata: {'total_pages': 44, 'file_path': '/Users/heewungsong/Experiment/Visa_Rag/Documents/E-9 FAQ_한국어.pdf', 'source': '1'}
Doc ID: 2c0b8083-a499-4f57-ac8c-2f1f6880fed1
Text: E-9 FAQ - 한국어 중학교졸업증만있는데한국에서일할수있어요? 고용허가제로한국에갈건데, 한국가서교육받을수있는지,
아니면미리자격증있어야 신청할수있는지알고싶어요. 예를들어서농업쪽에일하려고하는데농업에대한지식이 필요하나요?
한국에E9로가면숙박이나식사가제공되나요, 아니면스스로숙박찾아야되나요?
E9으로농업분야에서일하고싶은데100프로한국에가서일하는걸보장할수있나요? 올해추가등록이있는지물어봐도될까요? 2024년에E9
시험등록이더있을지알려주실수 있나요? 저여자인데한국에서식당에서일할수있나요? E9로한국에가려고하는데한국에서주로무슨업종이있나요?
저여자인데, 01년생이에요. E9 고용허가제에대해알아보고있는데여자이면가기어렵다는 걸맞나요? ...


Count of visa documents: 21
Visa metadata: {'total_pages': 21, 'file_path': '/Users/heewungsong/Experiment/Visa_Rag/Documents/E-9 Visa Guide_한국어.pdf', 'source': '1'}
Doc ID: 40278fb9-573e-4127-957c-8b397cc424eb
Text: E-9 Visa Guide - 한국어 고용허가제 고용허가제해당자와활동범위 고용허가제해당자의사증발급후1회체류기간의상한
고용허가제해당자의사증발급허용업종및체류자격약호(기호): E-9 근로자를채용할수있는 업종과범위
고용허가제해당자는사증발급인정서를받아야만E-9 사증을발급받을수있음
고용허가제해당자는2012.8.1부터범죄경력증명서및건강상태확인서제출 고용허가제비전문취업(E-9) 자격사증발급인정서발급절차
고용허

#### Document to Node

In [9]:
from llama_index.core.schema import TextNode

visa_nodes = []
faq_nodes = []

for faq_doc in faq_docs:
    if faq_doc.text:
        faq_nodes.append(TextNode(id_=faq_doc.id_, text=faq_doc.text))

for visa_doc in visa_docs:
    visa_nodes.append(TextNode(id_=visa_doc.id_, text=visa_doc.text))

#### Merge visa_nodes + faq_nodes

In [10]:
all_nodes = visa_nodes + faq_nodes

from llama_index.core import StorageContext
# initialize storage context (by default it's in-memory)

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(all_nodes)

#### Define Summary Index and Vector Index over Same Data

In [11]:

from llama_index.core import SummaryIndex
from llama_index.core import VectorStoreIndex

# Summary는 visa_nodes 만 사용
summary_index = SummaryIndex(visa_nodes, storage_context=storage_context)

# VectorStoreIndex는 all_nodes 사용
vector_index = VectorStoreIndex(all_nodes, storage_context=storage_context)

In [18]:
# from llama_index.core import VectorStoreIndex
# visa_vector_index = VectorStoreIndex(visa_nodes, storage_context=visa_storage_context)
# faq_vector_index = VectorStoreIndex(faq_nodes, storage_context=faq_storage_context)

#### Define Query Engines and Set Metadata 

Using CohereRarank

In [38]:
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.postprocessor import SimilarityPostprocessor

llm = OpenAI(model="gpt-4-0125-preview", temperature=0)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)


reranker = CohereRerank(
    top_n=3, model="rerank-multilingual-v2.0"
)

vector_query_engine = vector_index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[
        reranker,
        # SimilarityPostprocessor(similarity_cutoff=0.5)
    ]
)

# visa_vector_query_engine = visa_vector_index.as_query_engine(llm=llm)
# faq_vector_query_engine = faq_vector_index.as_query_engine(llm=llm)

In [39]:
summary_prompts = summary_query_engine.get_prompts()
vector_prompts = vector_query_engine.get_prompts()

print(f"Summary: {summary_prompts}\n\n\n")
print(f"Vector: {vector_prompts}")
# prompts['response_synthesizer:text_qa_template']

Summary: {'response_synthesizer:summary_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.SUMMARY: 'summary'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.SUMMARY: 'summary'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information from multiple sources is below.\n---------------------\n{context_str}\n---------------------\nGiven the information from multiple sources and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x17ccb54e0>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[Chat

#### Update Prompt Template

In [40]:
from llama_index.core import PromptTemplate
new_text_qa_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the query in Korean.\n"
    "Query: {query_str}\n"
    "Answer: "
)
new_qa_tmpl = PromptTemplate(new_text_qa_tmpl_str)

vector_query_engine.update_prompts({"response_synthesizer:text_qa_template": new_qa_tmpl})
# faq_vector_query_engine.update_prompts({"response_synthesizer:text_qa_template": new_qa_tmpl})

In [19]:
vector_prompts = vector_query_engine.get_prompts()
vector_prompts

{'response_synthesizer:summary_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.SUMMARY: 'summary'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.SUMMARY: 'summary'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information from multiple sources is below.\n---------------------\n{context_str}\n---------------------\nGiven the information from multiple sources and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x158ecb920>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(r

In [41]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to E-9 Korea Visa Guide."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from E-9 Korea Visa Guide and FAQ."
    ),
)

#### Define Router Query Engine
사용 가능한 선택기가 여러 개 있으며 각각 고유한 속성이 있습니다.

LLM 선택기는 LLM을 사용하여 구문 분석된 JSON을 출력하고 해당 인덱스를 쿼리합니다.

Pydantic 선택기(현재 gpt-4-0613및 gpt-3.5-turbo-0613(기본값)에서만 지원됨)는 원시 JSON을 구문 분석하는 대신 OpenAI Function Call API를 사용하여 Pydantic 선택 개체를 생성합니다.

각 선택기 유형에 대해 라우팅할 인덱스 1개 또는 여러 개를 선택하는 옵션도 있습니다.

PydanticSingleSelector
OpenAI Function API를 사용하여 라우터 선택기 내부에서 pydantic 개체를 생성/파싱합니다.

#### PydanticSingleSelector
OpenAI Function API를 사용하여 라우터 선택기 내부에서 pydantic 개체를 생성/파싱합니다.

In [42]:
from llama_index.core.query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors import (
    PydanticMultiSelector,
    PydanticSingleSelector,
)

# selector = PydanticSingleSelector.from_defaults(llm=OpenAI(model="gpt-4-0613"))
selector = PydanticMultiSelector.from_defaults(llm=OpenAI(model="gpt-4-0613"))
# selector = LLMMultiSelector.from_defaults(llm=OpenAI(model="gpt-4-0613"))


query_engine = RouterQueryEngine(
    selector=selector,
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
)

In [22]:
response = query_engine.query("저는 타투, 문신이 있는데 E9비자로 가도 되나요?")
print(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Selecting query engine 1: The question is asking for a specific context regarding tattoos and E-9 Korea Visa, which aligns with choice 2..


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

E-9 비전문취업비자는 타투 여부에 대한 명시적인 규정이 없습니다. 그러나 대한민국에서는 타투가 아직도 사회적으로 수용되지 않는 경우가 있을 수 있습니다. 따라서, 타투가 있는 경우 일부 고용주나 직장에서는 문제가 될 수 있으니 참고하시기 바랍니다. 타투 여부가 비자 심사에 직접적인 영향을 미치는 것은 아니지만, 실제 근무 환경에서 문제가 될 수 있습니다.


In [23]:
response = query_engine.query("저는 베트남사람이고 색맹인데 한국의 E-9 비자를 받을 수 있을까요?")
print(str(response))

Selecting query engine 1: The question is asking for a specific context regarding the eligibility of a color-blind person from Vietnam to receive an E-9 visa in Korea. Therefore, the second choice which is about retrieving specific context from E-9 Korea Visa information and FAQ is more relevant..


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

E-9 비자를 받기 위한 건강검진에는 일반적으로 시력 검사도 포함되어 있습니다. 색맹이 업무 수행에 지장을 주는 경우, 비자 발급이 어려울 수 있습니다. 그러나 색맹이 업무에 큰 영향을 미치지 않는다면, 비자를 받을 수 있을 가능성이 있습니다. 그러나 이는 개별 사례에 따라 다르므로, 신청 전에 한국 대사관이나 고용지원센터와 상의하는 것이 좋습니다.


In [35]:
langfuse_callback_handler.set_trace_params(
    name="임신",
    version="4",
)

# Q. 저 비자 E9인데 임신한다면 강제로 출국해야 하나요?
response = query_engine.query("저는 여자이고 현재 한국에 일하면서 체류하고 있는데 만약에 애를 갖게 되면 강제 송환될까요?")
print(str(response))

Selecting query engine 1: The question is asking for a specific context from the E-9 Korea Visa Guide and FAQ, particularly about the possibility of deportation if the questioner, a woman currently working and residing in Korea, becomes pregnant..
아니요, 한국에서 일하면서 아이를 가지게 되는 것만으로는 강제 송환되지 않습니다. 그러나 체류자격과 관련된 모든 사항은 출입국관리법 및 관련 규정에 따라 결정되므로, 상황에 따라 다를 수 있습니다. 자세한 사항은 출입국관리사무소에 문의하시는 것이 좋습니다.


In [43]:
langfuse_callback_handler.set_trace_params(
    name="고용허가제 선정국가에서 중국이 포함되어 있나요?",
    version="2",
)

response = query_engine.query("고용허가제 선정국가에서 중국이 포함되어 있나요?")
print(str(response))

Selecting query engine 1: The question is asking for a specific piece of information from the E-9 Korea Visa Guide, which choice 2 is most relevant for..
네, 중국은 고용허가제 선정국가에 포함되어 있습니다.


In [44]:
langfuse_callback_handler.set_trace_params(
    name="E9 비자의 재입국허가에 대해서 알려주세요.",
    version="1",
)

response = query_engine.query("E9 비자의 재입국허가에 대해서 알려주세요.")
print(str(response))

Selecting query engine 1: The question is asking for specific information about the re-entry permit of the E9 visa. This corresponds to choice 2 which is about retrieving specific context from the E-9 Korea Visa Guide and FAQ..
E-9 비자를 소지한 외국인 근로자가 재입국을 원할 경우, 재입국 특례제도를 이용할 수 있습니다. 이 제도는 일정한 요건을 갖춘 성실 외국인 근로자가 취업활동 기간이 만료되어 출국하기 전에 사용자가 재입국 후의 고용허가를 신청하면, 그 외국인 근로자에 대하여 출국한 날로부터 1개월이 지나면 다시 취업할 수 있는 특례입니다. 재입국 특례 대상자는 외국인 근로자의 고용 등에 관한 법률 제18조의 4 규정에 해당하는 자, 취업활동 기간 중에 사업장을 변경하지 않은 경우 또는 최초 근무한 업종과 동일 업종 내에서 사업장 변경하여 4년 10개월 근속한 경우, 사용자와 취업활동 기간 종료일까지의 근로 계약 기간이 1년 이상일 것 등의 조건을 충족해야 합니다. 재입국 허가 면제 조건도 있습니다. 등록을 필한 외국인이 출국한 날부터 1년 이내에 재입국하려는 경우 재입국 허가 면제, 체류 기간이 1년보다 적게 남아 있는 경우 체류 기간 범위 내에서 재입국 허가 면제 등이 있습니다.


In [45]:
langfuse_callback_handler.set_trace_params(
    name="고용허가제의 재입국허가에 대해서 알려주세요.",
    version="1-1",
)

response = query_engine.query("고용허가제의 재입국허가에 대해서 알려주세요.")
print(str(response))

Selecting query engine 1: The question is asking for specific information about re-entry permission under the Employment Permit System, which can be retrieved from the E-9 Korea Visa Guide and FAQ..
고용허가제에 해당하는 외국인근로자가 재입국을 원할 경우, 재입국 허가 면제 제도를 이용할 수 있습니다. 이는 등록을 필한 외국인이 출국한 날부터 1년 이내에 재입국하려는 경우, 또는 체류 기간이 1년보다 적게 남아 있는 경우에 해당됩니다. 단, 입국 규제 및 사증 발급 규제자는 체류지 관할 청에 방문하여 재입국 허가를 받아야 하며, 수수료를 제출해야 합니다. 

또한, 재입국 특례 제도를 이용할 수도 있습니다. 이는 일정한 요건을 갖춘 성실한 외국인 근로자가 취업 활동 기간이 만료되어 출국하기 전에 사용자가 재입국 후의 고용 허가를 신청하면, 그 외국인 근로자에 대하여 출국한 날로부터 1개월이 지나면 다시 취업할 수 있는 특례입니다. 이를 위해서는 외국인 근로자의 고용 등에 관한 법률 제18조의 4 규정에 해당하는 자이며, 취업 활동 기간 중에 사업장을 변경하지 않은 경우 또는 최초 근무한 업종과 동일 업종 내에서 사업장 변경하여 4년 10개월 근속한 경우, 그리고 사용자와 취업 활동 기간 종료일까지의 근로 계약 기간이 1년 이상일 것이 필요합니다.


In [48]:
query = "저는 베트남 사람이고 28살인데 한국에 가서 일할수 있나요?"

langfuse_callback_handler.set_trace_params(
    name=query,
    version="1",
)

response = query_engine.query(query)
print(str(response))

Selecting query engine 0: The question is asking for a summary of information related to eligibility for working in Korea, which is likely covered in the E-9 Korea Visa Guide..
Selecting query engine 1: The question may require specific context from the E-9 Korea Visa Guide and FAQ, particularly regarding the eligibility of a 28-year-old Vietnamese person to work in Korea..
Combining responses from multiple query engines.
네, 가능합니다. 베트남은 한국의 고용허가제 선정 국가 중 하나이므로 한국에서 일할 수 있습니다. 하지만, 한국에서 일하려면 E-9 비전문 취업 비자를 받아야 하며, 이를 위해 사증 발급 인정서, 범죄 경력 증명서, 건강 상태 확인서를 제출해야 합니다. 또한, 한국어능력시험 및 취업교육을 이수해야 하지만, 일부 경우에는 면제될 수 있습니다. EPS 시험을 치를 수 있는 나이는 39세까지이므로, 28세인 당신은 한국에서 일할 수 있습니다. 그러나, 한국에서 일하려면 특정 조건을 충족해야 하며, 이는 근로계약, 사업장 변경, 체류자격 변경 등 다양한 요소를 포함합니다. 이러한 요구사항들을 충족하면 한국에서 일할 수 있습니다.
